In [ ]:
import cPickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from time import time, sleep
from IPython import display
os.chdir(u'/Users/zbutler/research/fire_prediction')
from data import data
from geometry.grid_conversion import ak_bb, fairbanks_lat_lon, get_latlon_xy_fxns
from prediction.fire_clustering import cluster_over_time_with_merging, FIRE_SEASON
from util.daymonth import day2monthday
%matplotlib inline

In [ ]:
# Load all data
modis = data.load_modis() 
gfs_dict = data.load_gfs_dict()
clust_df = data.load_clust_df()
merge_dict = data.load_merge_dict()
min_year = np.min(modis.year)
max_year = np.max(modis.year)
clust_df.iloc[0:10]

In [ ]:
# Plot some merge statistics first
mergee_size = []
merger_size = []
for merger, (mergee, dayofyear) in merge_dict.iteritems():
    mergee_size.append(len(clust_df[(clust_df.cluster==mergee) & (clust_df.dayofyear < dayofyear)]))
    merger_size.append(len(clust_df[(clust_df.cluster==merger) & (clust_df.dayofyear < dayofyear)]))
plt.hist(merger_size, bins=100)
plt.xlabel("Merger size")
plt.savefig("pics/merger_size_10_hist.png")
plt.show()
plt.close()
plt.hist(mergee_size, bins=100)
plt.xlabel("Mergee size")
plt.savefig("pics/mergee_size_10_hist.png")
plt.show()

In [ ]:
# Find a couple merges that might be interesting to plot
for merger, (mergee, dayofyear) in merge_dict.iteritems():
    mergee_size = len(clust_df[(clust_df.cluster==mergee) & (clust_df.dayofyear < dayofyear)])
    if mergee_size > 1000:
        print "mergee %d (size %d, year %d)" % (mergee, mergee_size, clust_df[(clust_df.cluster==mergee)].year.unique()[0]),
        print "merger %d (size %d)" % (merger, len(clust_df[(clust_df.cluster==merger) & (clust_df.dayofyear < dayofyear)]))
        print "lat lon: " + str((np.mean(clust_df[(clust_df.cluster==merger)].lat), np.mean(clust_df[(clust_df.cluster==merger)].long)))


In [ ]:
# Let's check out monster merger 917/961
from util.daymonth import monthday2day
mergee = 917
merger = 961
mergee_df = clust_df[clust_df.cluster == mergee]
merger_df = clust_df[clust_df.cluster == merger]
year = mergee_df.year.unique()[0]
min_day = min(np.min(mergee_df.dayofyear), np.min(merger_df.dayofyear))
max_day = np.max(mergee_df.dayofyear)

# Get size in grid
min_x = min(np.min(mergee_df.x), np.min(merger_df.x))
max_x = max(np.max(mergee_df.x), np.max(merger_df.x))
min_y = min(np.min(mergee_df.y), np.min(merger_df.y))
max_y = max(np.max(mergee_df.y), np.max(merger_df.y))
prev_merger_xs = []
prev_mergee_xs = []
prev_merger_ys = []
prev_mergee_ys = []
day = monthday2day(8,1)
for dayofyear in xrange(int(min_day), int(merge_dict[merger][1])+2):
    merger_fires = merger_df[merger_df.dayofyear == dayofyear]
    mergee_fires = mergee_df[mergee_df.dayofyear == dayofyear]
    display.clear_output(wait=True)
#    plt.figure(figsize=(5,10))
    axes = plt.gca()
    axes.set_xlim([min_x,max_x])
    axes.set_ylim([min_y,max_y])
    if len(prev_merger_xs):
        plt.plot(prev_merger_xs, prev_merger_ys, 'ro')
    if len(prev_mergee_xs):
        plt.plot(prev_mergee_xs, prev_mergee_ys, 'bo')

    if len(merger_fires):
        today_xs, today_ys = (np.array(merger_fires.x), np.array(merger_fires.y))
        plt.plot(today_xs, today_ys, 'r*')
        prev_merger_xs += list(today_xs)
        prev_merger_ys += list(today_ys)
    if len(mergee_fires):
        today_xs, today_ys = (np.array(mergee_fires.x), np.array(mergee_fires.y))
        plt.plot(today_xs, today_ys, 'b*')
        prev_mergee_xs += list(today_xs)
        prev_mergee_ys += list(today_ys)
    month, day = day2monthday(dayofyear, (year % 4) == 0)
    plt.title("Fires on %d %d" %(month, day))
    plt.show()
    sleep(1)
    dayofyear += 1